# DCA in backtest
Here to more simulate the trading, we use backtest module and implement a DCA strategy to verify it.
The goodness of `class Market` is we only need to care about how to react with it.  This allow user simulate more complicate trading operation like
sending order, cancel order,...etc

In [3]:
# Run it when crypto-backtest is updated, and then restart the kernel
!pip uninstall crypto-backtest -y
!pip install git+https://github.com/BullionBear/crypto-backtest.git

Found existing installation: crypto-backtest 0.0.5
Uninstalling crypto-backtest-0.0.5:
  Successfully uninstalled crypto-backtest-0.0.5
  Cloning https://github.com/BullionBear/crypto-backtest.git to /tmp/pip-req-build-rxc2n6dk
  Running command git clone --filter=blob:none --quiet https://github.com/BullionBear/crypto-backtest.git /tmp/pip-req-build-rxc2n6dk
  Resolved https://github.com/BullionBear/crypto-backtest.git to commit 2f5d3998f72eeb831eb8da0d978971b167447102
  Preparing metadata (setup.py) ... done
  Created wheel for crypto-backtest: filename=crypto_backtest-0.0.6-py3-none-any.whl size=9332 sha256=297a3348a9b03236eaec5feb4eeffacab644e8407f73b20514fdc195f533b32a
  Stored in directory: /tmp/pip-ephem-wheel-cache-slvdz2vi/wheels/14/d1/5f/193347d44b51670dd287000ac535cdb7a8fc3f84e426532382
Successfully built crypto-backtest

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import backtest
backtest.__version__

'develop'

In [4]:
from backtest.components.core import KLineIteratorFromFileSystem
from backtest.components import KLineMarket

from datetime import datetime, timezone

start_time = datetime(2023, 3, 8, tzinfo=timezone.utc).timestamp() * 1000
end_time = datetime(2023, 4, 8, tzinfo=timezone.utc).timestamp() * 1000
data_source = '/home/yite/crypto_data/binance/data'
kline_iterator = KLineIteratorFromFileSystem('BTCUSDT', start_time, end_time, data_source)
market = KLineMarket(kline_iterator)

In [5]:
market.get_ts()

22245.86